In [10]:
'''Our first step is to identify the URLs we want to download. We are going to center our search in the most popular 
job posting site: Indeed. Our main objective is to create a spreadsheet of jobs that fits into categories input 
by the user, like salary, location whether it is remote or not, etc, that could be helpful in defining a job search'''

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
import json
import os
import requests
from urllib.parse import urljoin
import re

#First step is understanding the indeed url infraestructure
companies = {}
job_titles = []
urls = []
salaries = []
job_attributes = []
companies_info = {}
page_results = ''
URL = 'https://www.indeed.com/jobs?q=Data+Analyst&jt=fulltime&limit=50&sr=directhire&fromage=1'

indeed = requests.get(URL)
indeed_content = indeed.content
parser = BeautifulSoup(indeed_content, 'html.parser')
results = parser.find_all('div', attrs={'id': "searchCountPages"})
for x in results:
    print(x.text)
    page_results = x.text[-8:-5]
    print(page_results)
        
        
page_results = round((int(page_results.replace(',',''))/50)+2)
print(page_results)
for i in range(0, page_results):
    if i == 0:
        url = 'https://www.indeed.com/jobs?q=Data+Analyst&jt=fulltime&limit=50&sr=directhire&fromage=1'
        indeed = requests.get(url)
        indeed_content = indeed.content
        parser = BeautifulSoup(indeed_content, 'html.parser')
        results = parser.find_all('div', attrs={'data-tn-component': 'organicJob'})
        for x in results:
            job_attributes = []
            company = x.find('span', attrs={"class":"company"}).text
            if company:
                company = company.replace('\n', '').strip()
            location = x.find('span', attrs={"class":"location accessible-contrast-color-location"})
            if location:
                job_attributes.append((location.text).replace('\n', '').strip())
            else:
                job_attributes.append('Not Especified')
            job = x.find('a', attrs={'data-tn-element': "jobTitle"}).text
            if job:
                job = job.replace('\n', '').strip()
                job_attributes.append(job)
            salary = x.find('span', class_ = 'salaryText')
            if salary:
                job_attributes.append((salary.text).replace('\n', '').strip())
            else:
                job_attributes.append('Not Especified')
            remote = x.find('span', class_ = 'remote')
            if remote:
                job_attributes.append((remote.text).replace('\n', '').strip())
            else:
                job_attributes.append('No')
            rating = x.find('span', class_ = 'ratingsContent')
            if rating:
                job_attributes.append((rating.text).replace('\n', '').strip())
            else:
                job_attributes.append('-')
            companies[company] = job_attributes
            job_attributes = []
    else:
        url = 'https://www.indeed.com/jobs?q=Data+Analyst&jt=fulltime&limit=50&sr=directhire&fromage=1&start={}'.format(i*50)
        indeed = requests.get(url)
        indeed_content = indeed.content
        parser = BeautifulSoup(indeed_content, 'html.parser')
        results = parser.find_all('div', attrs={'data-tn-component': 'organicJob'})
        for x in results:
            job_attributes = []
            company = x.find('span', class_ = "company").text
            if company not in companies:
                company = company.replace('\n', '').strip()
            else:
                if 'Repeated Companies' in companies:
                    companies['Repeated Companies'] += 1
                else:
                    companies['Repeated Companies'] = 1
                    next
            location = x.find('span', attrs={"class":"location accessible-contrast-color-location"})
            if location:
                job_attributes.append((location.text).replace('\n', '').strip())
            else:
                job_attributes.append('Not Especified')
            job = x.find('a', attrs={'data-tn-element': "jobTitle"}).text
            if job:
                job = job.replace('\n', '').strip()
                job_attributes.append(job)
            salary = x.find('span', class_ = 'salaryText')
            if salary:
                job_attributes.append((salary.text).replace('\n', '').strip())
            else:
                job_attributes.append('Not Especified')
            remote = x.find('span', attrs = {'class' : 'remote'})
            if remote:
                job_attributes.append((remote.text).replace('\n', '').strip())
            else:
                job_attributes.append('No')
            rating = x.find('span', class_ = 'ratingsContent')
            if rating:
                job_attributes.append((rating.text).replace('\n', '').strip())
            else:
                job_attributes.append('-')
            companies[company] = job_attributes
            job_attributes = []
    


print(companies)


                    Page 1 of 436 jobs
436
11
{'ArcheMedX': ['Not Especified', 'Experienced Data Analyst (Looker)', 'Not Especified', 'Remote', '-'], 'Valleysoft Technologies': ['New York, NY', 'Junior BI Analyst(US Citizens and GC only)', '$60,000 - $80,000 a year', 'No', '-'], 'New York City DEPT OF HEALTH/MENTAL HYGIENE': ['New York, NY', 'Data Analyst, Public Health Laboratory', '$64,140 - $77,609 a year', 'No', '4.1'], 'OnlineMedEd': ['Austin, TX 78758 (North Burnett area)', 'Data Analyst - People Analytics', '$70,000 - $90,000 a year', 'No', '-'], 'VISITING NURSE SERVICE OF NEW YORK': ['Manhattan, NY', 'Claims Systems Configuration Analyst', 'Not Especified', 'No', '3.6'], 'Klyxx Creative': ['New York, NY', 'Growth Marketing Analyst', '$45,000 a year', 'Temporarily remote', '-'], 'BNY Mellon': ['Pittsburgh, PA', 'Data Engineer', 'Not Especified', 'No', '3.5'], 'Citi': ['Tampa, FL', 'Data Quality Lead Analyst (VP)', 'Not Especified', 'No', '3.9'], 'AETNA': ['Hartford, CT 06156 (A

In [12]:
df = pd.DataFrame.from_dict(companies, orient = 'index').reset_index()
df.columns = ['Company', 'Location', 'Title', 'Salary', 'Remote?', 'Rating']
df.shape

(114, 6)